In [14]:
import sys
import numpy as np
import random

UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

In [2]:
'''
Enumerate states for a 3x3 grid ==> 81 states 
(9 choices for pacman location x 9 choices for ghost location)

y    _0_|_1_|_2_
|    _3_|_4_|_5_
v     6 | 7 | 8
    x -->

Each x,y pair represented as an integer number corresponding to the diagram above
'''

states_num = [];

for s in range(81):
    for p in range(9):
        for g in range(9):
            states_num.append( (p, g) )
                    
#for s in range(81):
#    print("state ", s, ": ", states_num[s])

In [3]:
def grid_to_xy(number):
    switch = {
        0: [0,0],
        1: [1,0],
        2: [2,0],
        3: [0,1],
        4: [1,1],
        5: [2,1],
        6: [0,2],
        7: [1,2],
        8: [2,2]
    }
    return switch.get(number, "invalid entry")

def xy_to_grid(x,y):
    switch = {
        0: {0:0, 1:3, 2:6},
        1: {0:1, 1:4, 2:7},
        2: {0:2, 1:5, 2:8}
    }
    x = switch.get(x,"invalid entry")

    if x == "invalid entry":
        return x
    else:
        return x.get(y,"invalid entry")

def return_state(states, p, g):
    return states.index( (p,g) )

def move(x, y, action):
    if action == UP:
        y = max(0, y-1)
    elif action == RIGHT:
        x = min(2, x+1)
    elif action == DOWN:
        y = min(2, y+1)
    elif action == LEFT:
        x = max(0, x-1)
    return xy_to_grid(x, y)

In [4]:
class PacmanEnv:
    '''
    Class to initialize and store information about the Pacman environment for program planning algorithms.

    Properties:
        P[s][a] is a list of is a list of transition tuples (prob, next_state, reward, done)
        num_states = number of states (set to default for 3x3 grid)
        num_actions = number of actions (set to 4)
        pellet_loc = location of pellet (set to 2, i.e. [2,0] by default)

    Methods:
        return_state: Returns state number given location of pacman and the ghost
        move: Moves pacman given current location and action input. Returns grid location number
        calculate_reward: Returns reward for current location of pacman. Used to evaluate R(s,a,s') by 
                        first determining s' through move(s,a), then calculating the reward at s'.
        grid_to_xy: Returns corresponding (x,y) coordinate pair for valid grid location integer input
                    If number out of range, returns 'invalid entry' error message
        xy_to_grid: Returns corresponding grid location # for given (x,y) coordinate pair input
                    If number out of range, returns 'invalid entry' error message
    '''

    def __init__(self, states=states_num, num_states=81, num_actions=4, pellet_loc=2):
        self.states = states
        self.num_states = num_states
        self.num_actions = num_actions
        self.pellet_loc = pellet_loc
        
        P = {s : {a : [] for a in range(num_actions)} for s in range(num_states)}
        
        # parameters must be of the same type, i.e. [x,y] or int value 0-8
        # need to adjust to include reward definition for bumping into walls
        def calculate_reward(pacman_new_loc, ghost_new_loc, ghost_current_loc, pellet_location):
            if pacman_new_loc == ghost_current_loc: # pacman moved to the ghost's location
                return -1000
            elif pacman_new_loc == pellet_location:
                return 1000
            elif pacman_new_loc == ghost_new_loc: # the ghost moved to pacman's new location
                return -1000
            else:
                return 0
        
        for s in range(num_states):
            for pacman_a in range(num_actions):
                done = False # flag to signal game has ended
                temp = P[s][pacman_a]
                pacman_grid_loc = states[s][0] # for the given state, where is pacman
                ghost_grid_loc = states[s][1] # in the given state, where is the ghost
                
                # if pacman performs action a: 0=UP, 1=RIGHT, 2=DOWN, 3=LEFT
                [x_p, y_p] = grid_to_xy(pacman_grid_loc)
                next_pacman_loc = move(x_p, y_p, pacman_a) # grid location he will move to
                
                for ghost_a in range(num_actions):
                    # if the ghost performs action a: 0=UP, 1=RIGHT, 2=DOWN, 3=LEFT
                    [x_g, y_g] = grid_to_xy(ghost_grid_loc)
                    next_ghost_loc = move(x_g, y_g, ghost_a) # grid location he will move to
                    
                    # resulting next state, simulates pacman and the ghost moving simultaneously
                    next_state = return_state(states, next_pacman_loc, next_ghost_loc) 
                    reward = calculate_reward(next_pacman_loc, next_ghost_loc, ghost_grid_loc, pellet_loc) # calculate the reward at this state

                    if (pacman_grid_loc == pellet_loc or pacman_grid_loc == ghost_grid_loc):
                        done = True

                    temp.append( (0.25, next_state, reward, done) )
        
        self.P = P
        

In [5]:
def value_iteration(env=PacmanEnv(), gamma=0.9, theta=1e-5):
    '''
    Value Iteration Algorithm

    Inputs:
        env: PacmanEnv as defined in class above.
        gamma: Discount rate for future rewards.
        theta: Stopping criterion value. When change in Value function is less than theta for every state, stop.

    Helper Methods:
        calculate_action_values: Calculates the values for all actions for a given state.
                                Returns a vector action_values of length num_actions, where 
                                action_values[a] = expected value of action a.
                                The expected value is calculated according to the Bellman equation:
                                V(s) = P(s'|s,a) * ( R(s,a) + (gamma * V(s')) )
        extract_policy: Returns the optimal policy for a given value function. It is run once at the end of the algorithm
                        after the optimal V (value function) has been calculated.

    Outputs:
        A tuple (policy, V, steps) of the optimal policy, the approximated optimal value function, and the number of steps
        the algorithm took to converge.
    '''
    
    def calculate_action_values(current_state, V):
        action_values = np.zeros(env.num_actions)
        for a in range(env.num_actions):
            for prob, next_state, reward, done in env.P[current_state][a]:
                action_values[a] += prob * (reward + (gamma * V[next_state]))
        return action_values
    
    def extract_policy(V):
        policy = np.zeros([env.num_states, env.num_actions])
        
        for s in range(env.num_states):
            action_values = calculate_action_values(s, V)
            best_action = np.argmax(action_values) # returns index of action that has maximum V
            policy[s, best_action] = 1 # deterministic optimal policy, i.e. always take best_action for given state
        
        return policy
    
    V = np.zeros(env.num_states) # arbitrarily initialize vector V to be all zeros
    converged = False
    steps = 0
    
    # iteratively calculate optimal V
    while not converged:
        print('Value iteration, step ', steps, '...')
        delta = 0
        for s in range(env.num_states):
            action_values = calculate_action_values(s, V)
            max_action_value = np.max(action_values)
            delta = max( delta, np.abs(max_action_value - V[s]) ) # the maximum difference between V'(s) and V(s) for all s
            V[s] = max_action_value        
        
        steps += 1
        
        #print('Delta: ', delta)
        converged = (delta < theta)
        #print(converged)
    
    # extract optimal policy after calculating optimal V
    policy = extract_policy(V)
    
    return policy, V, steps

In [6]:
import math
winReward = 1000
loseReward = -1000

def nextMove(env, currState, action):
    '''
    Returns all possible next states given action, and the reward given next states.
    ARGS:
        currState: Current game state, ranges from 0 to numStates-1.
        action: Action to perform.
    RETURN:
        nextStates: List of possible next states give current state and action.
                    Largly depends on potential ghost movements.
        rewards: List of rewards for each potential next state.
    '''
    nextStates = [];
    # Get currState coordinates
    
    pacman_grid_loc = env.states[currState][0] # for the given state, where is pacman
    ghost_grid_loc = env.states[currState][1] # in the given state, where is the ghost
                
    [x_p, y_p] = grid_to_xy(pacman_grid_loc)
    [x_g, y_g] = grid_to_xy(ghost_grid_loc)
    
    # Get pacman location after performing action
    next_pacman_loc = move(x_p, y_p, action)
    pacmanLocX_next, pacmanLocY_next = grid_to_xy(next_pacman_loc)
    
    # Initialize rewards
    if (next_pacman_loc == env.pellet_loc): 
        rewards = [winReward]*4;
    elif (next_pacman_loc == ghost_grid_loc):
        rewards = [loseReward]*4;
    else:
        rewards = [0]*4;

    # Iterate through possible ghost states
    for ghostAction in range(4):
        next_ghost_loc = move(x_g, y_g, ghostAction)
        ghostLocX_next, ghostLocY_next = grid_to_xy(next_ghost_loc)
        state = return_state(env.states, next_pacman_loc, next_ghost_loc)
        nextStates.append(state);     
        # Evaluate reward - pacman eaten by ghost
        if (next_pacman_loc == next_ghost_loc):
            rewards[ghostAction] = loseReward;

    return nextStates, rewards;

In [7]:
import math
winReward = 1000
loseReward = -1000

###############################################################################
############################  Policy Iteration ################################
###############################################################################
def policy_iteration(env=PacmanEnv(), deltaLim=1e-5, gamma=0.9):
    '''
    Policy iteration algorithm consists of two steps that are repeated in each iteration of the loop:
    policy evaluation (calculate V) and policy improvement (greedy approach to improve current policy).
    
    Helper Methods:
        policyEvaluation
        policyImprovement
    '''
    prob = 0.25
    
    ########################## Policy evaluation ##################################
    def policyEvaluation(policy, v, deltaLim=1e-5, delta=10000):
        cnt = 0;
        while (delta > deltaLim): 
            delta = 0;
            for state in range(env.num_states):
                if (env.P[state][0][0][3] == True): # game has ended
                    v[state] = 0;
                    continue;
                vStateOld = v[state]; 
                # Choose action from policy
                action = policy[state];
                # Potential next states given current state and action
                nextStates, rewards = nextMove(env, state, action); 
                # Compute V(s)
                vNextStates = [prob*(rewards[i]+gamma*v[nextStates[i]]) for i in range(len(nextStates))];
                v[state] = sum(vNextStates);

                delta = max(delta, abs(v[state]-vStateOld));
            print("Delta",delta);
            cnt += 1;
        print("Policy evalulation converged at", cnt);
        return v;


    ########################## Policy Improvement #################################
    def policyImprovement(policy, v):
        policyStable = True;
        for state in range(env.num_states):
            if (env.P[state][0][0][3] == True): # game ended
                continue;
            oldAction = policy[state];
            # Optimal action and V(s,a)
            optAction = 0; 
            optActionVal = None;
            for action in range(4):
                # Potential next states given current state and action
                nextStates, rewards = nextMove(env, state, action);
                # Compute V(s,a)
                vNextStates = [prob*(rewards[i]+gamma*v[nextStates[i]]) for i in range(len(nextStates))];
                actionVal = sum(vNextStates);
                if (optActionVal == None):
                    optActionVal = actionVal;
                if (actionVal > optActionVal): # Choose action with largest V(s,a)
                    optActionVal = actionVal;
                    optAction = action;
            # No convergence
            if (oldAction != optAction):
                policyStable = False;
                policy[state] = optAction;
        return policy, policyStable
    
    # Init
    v = [0]*env.num_states;
    policy = [0]*env.num_states;
    policyStable = False;

    # Run Policy Iteration
    policyIter = 1;
    while (not policyStable):
        v = policyEvaluation(policy, v);
        policy, policyStable = policyImprovement(policy, v);
        # print("Total iterations:", policyIter)
        policyIter += 1;

    return policy, v, policyIter

In [8]:
policy, V, steps = policy_iteration()

Delta 1345.9564041748047
Delta 138.45322265624998
Delta 80.36209533691408
Delta 50.327251831054696
Delta 33.73082389033124
Delta 27.188297693384726
Delta 21.966299296992872
Delta 17.57000472180266
Delta 13.958241635996899
Delta 11.036803885005725
Delta 8.69804945493371
Delta 6.838899352821443
Delta 5.368187924993606
Delta 4.20872259239178
Delta 3.296839179868016
Delta 2.5809058406553618
Delta 2.0195120096609003
Delta 1.5796944540835511
Delta 1.2353503818944773
Delta 0.9658848837300411
Delta 0.7550907005195313
Delta 0.5902370677873137
Delta 0.461337393511144
Delta 0.36056539611337257
Delta 0.28325655650687054
Delta 0.22723088283598258
Delta 0.18228629704020705
Delta 0.1462312642899235
Delta 0.11730760240138238
Delta 0.09410482672890907
Delta 0.0754914077958233
Delta 0.06055961086141792
Delta 0.04858123833491845
Delta 0.038972122246548224
Delta 0.0312636379597393
Delta 0.025079851325187974
Delta 0.020119185502721848
Delta 0.01613971373265599
Delta 0.012947360991546475
Delta 0.01038643923

In [41]:
def q_learning(env=PacmanEnv(), gamma=0.9, alpha=0.9, epsilon=0.1, episodes=5):
    '''
    Q-Learning Algorithm

    Inputs:
        env: PacmanEnv as defined in class above. This will be used for simplicity of implementation, however in the case
            of Q-learning, the state transition probability matrix is assumed to be unknown.
        gamma: Discount rate for future rewards.
        alpha: Learning rate. "How much you accept the new value vs the old value," i.e. how much weight will you assign
            to the old vs new value of Q.
        epsilon: Used to control balance of exploration (choose a random action) vs exploitation, i.e. we pick a value at
            random in the range (0,1) and if this value < epsilon, we will choose a random action. Else, we pick the action
            that maximizes Q (based on current knowledge of Q).
        episodes: Number of epochs to run.

    Helper Methods:
        extract_policy: Returns the optimal policy for a given value function. It is run once at the end of the algorithm
                        after the optimal Q (value function) has been estimated.

    Outputs:
        A tuple (policy, Q, steps) of the policy extracted from the estimated Q function, the approximated optimal value 
        function, and the number of steps the algorithm took to converge.
    '''
    
    def extract_policy(Q):
        policy = np.zeros([env.num_states, env.num_actions])
        
        for s in range(env.num_states):
            best_action = np.argmax(Q[s, :]) # returns index of action that has maximum V
            policy[s, best_action] = 1 # deterministic optimal policy, i.e. always take best_action for given state
        
        return policy
    
    # initialize Q(s,a) matrix to all zeros
    Q = np.zeros([env.num_states, env.num_actions])
    steps = 0
    
    for t in range(episodes):
        print('Episode #', t)
        converged = False
        
         # select random state
        state = random.randint(0, env.num_states-1)
        
        # run inner loop for each episode until a terminal state has been reached
        while not converged:
            #print('Q learning, step ', steps, '...')
            
            # select action
            if random.uniform(0, 1) < epsilon:
                action = random.randint(0,3) # exploration
                #print('random action: ', action)
            else:
                action = np.argmax(Q[state, :]) # exploitation
                #print('exploit action: ', action)

            ghost_mvmt = random.randint(0,3) # simulate random movement for the ghost

            # travel to the next state, taking action selected above
            pacman_grid_loc = env.states[state][0] # for the given state, where is pacman
            ghost_grid_loc = env.states[state][1] # in the given state, where is the ghost

            [x_p, y_p] = grid_to_xy(pacman_grid_loc)
            next_pacman_loc = move(x_p, y_p, action)
            [x_g, y_g] = grid_to_xy(ghost_grid_loc)
            next_ghost_loc = move(x_g, y_g, ghost_mvmt)

            next_state = return_state(env.states, next_pacman_loc, next_ghost_loc)
            #print('next state: ', next_state)
        
            # to get reward, need to find specific entry of P[s][a] with same next_state...
            reward = 0
            for ns in range( len(env.P[state][action]) ): # array of tuples (probability, next_state, reward, done)
                if (env.P[state][action][ns][1] == next_state):
                    reward = env.P[state][action][ns][2]
            #print('reward: ', reward)

            # in next state, select action with highest Q-value
            max_next_action_value = np.max(Q[next_state, :])

            # update Q-values tables with equation
            #print('old value: ', Q[state][action])
            Q[state][action] = ((1-alpha)*Q[state][action]) + (alpha*(reward + (gamma * max_next_action_value)))
            #print('new value: ', Q[state][action])

            # set next state as current state & repeat
            state = next_state 
            steps += 1
            
            # if reached terminal state (i.e. next state = terminal state), converged = True
            #print('convergence: ', env.P[next_state][action][0][3])
            converged = env.P[next_state][action][0][3]
            #print('converged: ', converged)

    # extract optimal policy after calculating optimal V
    policy = extract_policy(Q)

    return policy, Q, steps

In [42]:
### Value Iteration Result ###
def convert_to_action(num):
    switch = {
                0: 'UP',
                1: 'RIGHT',
                2: 'DOWN',
                3: 'LEFT'
            }
    return switch.get(num, "invalid entry")

policy, V, steps = value_iteration(gamma=0.5)
env = PacmanEnv()
action_list = []

print('-----------------------------------------')
print('Optimal extracted policy: ')
print(policy)

print('-----------------------------------------')
for s in range(env.num_states):
    print()
    print('Location of Pacman: ', grid_to_xy(env.states[s][0]))
    print('Location of Ghost: ', grid_to_xy(env.states[s][1]))
    print('Best action determined by optimal policy: ', convert_to_action( policy[s].tolist().index(1) ) )
    action_list.append(convert_to_action( policy[s].tolist().index(1) ))

print('-----------------------------------------')    
print('Value Function: ')
print(V)

print('Steps \'til convergence: ', steps)
print('-----------------------------------------')

Value iteration, step  0 ...
Value iteration, step  1 ...
Value iteration, step  2 ...
Value iteration, step  3 ...
Value iteration, step  4 ...
Value iteration, step  5 ...
Value iteration, step  6 ...
Value iteration, step  7 ...
Value iteration, step  8 ...
Value iteration, step  9 ...
Value iteration, step  10 ...
Value iteration, step  11 ...
Value iteration, step  12 ...
Value iteration, step  13 ...
Value iteration, step  14 ...
Value iteration, step  15 ...
Value iteration, step  16 ...
Value iteration, step  17 ...
Value iteration, step  18 ...
Value iteration, step  19 ...
Value iteration, step  20 ...
-----------------------------------------
Optimal extracted policy: 
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 

In [43]:
### Policy iteration result ###
policy2, V2, steps2 = policy_iteration(gamma=0.5)
env = PacmanEnv()
action_list2 = []

print('-----------------------------------------')
print('Optimal extracted policy: ')
print(policy2)

print('-----------------------------------------')
for s in range(env.num_states):
    print()
    print('Location of Pacman: ', grid_to_xy(env.states[s][0]))
    print('Location of Ghost: ', grid_to_xy(env.states[s][1]))
    print('Best action determined by optimal policy: ', convert_to_action( policy2[s] ) )
    action_list2.append(convert_to_action( policy2[s] ))

print('-----------------------------------------')    
print('Value Function: ')
print(V2)

print('Steps \'til convergence: ', steps2)
print('-----------------------------------------')

Delta 1158.9641571044922
Delta 70.37353515625
Delta 22.15576171875
Delta 7.0171356201171875
Delta 2.2110939025878906
Delta 0.7046712562441826
Delta 0.2284294314449653
Delta 0.0753918632199202
Delta 0.025302088840135184
Delta 0.008614999952101243
Delta 0.0029681235470206957
Delta 0.0010321105410753262
Delta 0.0003614107556018098
Delta 0.0001271984433515172
Delta 4.4927232920599636e-05
Delta 1.5906580870250764e-05
Delta 5.640310405397031e-06
Policy evalulation converged at 17
Delta 1434.5930225674501
Delta 593.0232546375245
Delta 83.82783805393524
Delta 9.83109652231062
Delta 2.2626809498884084
Delta 0.7660799726641727
Delta 0.25537548775460905
Delta 0.08474518926699659
Delta 0.02807864839445351
Delta 0.009298309990644071
Delta 0.003078785205561374
Delta 0.0010195006612541135
Delta 0.00033765196742407966
Delta 0.00011185193935148163
Delta 3.7061030845109144e-05
Delta 1.2282651917772114e-05
Delta 4.071596892529783e-06
Policy evalulation converged at 17
Delta 420.77652488751005
Delta 116.0

In [37]:
### Comparison of value iteration optimal policy and policy iteration optimal policy ###

diff_opt_states = []

for s in range(env.num_states):
    if (action_list[s] != action_list2[s]):
        diff_opt_states.append(s)

count_diff = 0
for i in range(len(diff_opt_states)):
    if (action_list[i] != action_list2[i]):
        print('---------Different at state ', i, '---------')
        print('Value iteration optimal policy: ',action_list[i])
        print('Policy iteration optimal policy: ',action_list2[i])
        count_diff += 1

print('***************************************************************************')
print('Number of differences in policy: ', count_diff)

---------Different at state  0 ---------
Value iteration optimal policy:  RIGHT
Policy iteration optimal policy:  UP
---------Different at state  2 ---------
Value iteration optimal policy:  RIGHT
Policy iteration optimal policy:  UP
---------Different at state  4 ---------
Value iteration optimal policy:  RIGHT
Policy iteration optimal policy:  UP
---------Different at state  10 ---------
Value iteration optimal policy:  RIGHT
Policy iteration optimal policy:  UP
***************************************************************************
Number of differences in policy:  4


In [28]:
### Q-Learning result for episodes = 1000 ###
policy3, Q, steps3 = q_learning(episodes=1000)
env = PacmanEnv()
action_list3 = []

print('-----------------------------------------')
print('Extracted policy: ')
print(policy3)

print('-----------------------------------------')
for s in range(env.num_states):
    print()
    print('Location of Pacman: ', grid_to_xy(env.states[s][0]))
    print('Location of Ghost: ', grid_to_xy(env.states[s][1]))
    print('Best action determined by optimal policy: ', convert_to_action( policy3[s].tolist().index(1) ) )
    action_list3.append(convert_to_action( policy3[s].tolist().index(1) ))

print('-----------------------------------------')    
print('Value Function: ')
print(Q)
print('-----------------------------------------') 

print('Steps \'til convergence: ', steps3)
print('Average steps per episode: ', steps3/1000)
print('-----------------------------------------')

Episode # 0
Q learning, step  0 ...
Q learning, step  1 ...
Q learning, step  2 ...
Q learning, step  3 ...
Q learning, step  4 ...
Q learning, step  5 ...
Q learning, step  6 ...
Q learning, step  7 ...
Q learning, step  8 ...
Q learning, step  9 ...
Q learning, step  10 ...
Q learning, step  11 ...
Q learning, step  12 ...
Q learning, step  13 ...
Episode # 1
Q learning, step  14 ...
Q learning, step  15 ...
Q learning, step  16 ...
Q learning, step  17 ...
Q learning, step  18 ...
Q learning, step  19 ...
Q learning, step  20 ...
Q learning, step  21 ...
Episode # 2
Q learning, step  22 ...
Q learning, step  23 ...
Q learning, step  24 ...
Episode # 3
Q learning, step  25 ...
Episode # 4
Q learning, step  26 ...
Q learning, step  27 ...
Q learning, step  28 ...
Q learning, step  29 ...
Q learning, step  30 ...
Q learning, step  31 ...
Q learning, step  32 ...
Q learning, step  33 ...
Q learning, step  34 ...
Q learning, step  35 ...
Q learning, step  36 ...
Q learning, step  37 ...


Q learning, step  490 ...
Q learning, step  491 ...
Q learning, step  492 ...
Q learning, step  493 ...
Q learning, step  494 ...
Episode # 134
Q learning, step  495 ...
Q learning, step  496 ...
Q learning, step  497 ...
Q learning, step  498 ...
Q learning, step  499 ...
Q learning, step  500 ...
Q learning, step  501 ...
Episode # 135
Q learning, step  502 ...
Episode # 136
Q learning, step  503 ...
Episode # 137
Q learning, step  504 ...
Q learning, step  505 ...
Q learning, step  506 ...
Q learning, step  507 ...
Q learning, step  508 ...
Q learning, step  509 ...
Episode # 138
Q learning, step  510 ...
Episode # 139
Q learning, step  511 ...
Q learning, step  512 ...
Q learning, step  513 ...
Q learning, step  514 ...
Episode # 140
Q learning, step  515 ...
Q learning, step  516 ...
Q learning, step  517 ...
Episode # 141
Q learning, step  518 ...
Q learning, step  519 ...
Q learning, step  520 ...
Q learning, step  521 ...
Q learning, step  522 ...
Q learning, step  523 ...
Q le

Q learning, step  859 ...
Episode # 229
Q learning, step  860 ...
Q learning, step  861 ...
Q learning, step  862 ...
Q learning, step  863 ...
Q learning, step  864 ...
Q learning, step  865 ...
Q learning, step  866 ...
Q learning, step  867 ...
Q learning, step  868 ...
Q learning, step  869 ...
Q learning, step  870 ...
Q learning, step  871 ...
Q learning, step  872 ...
Episode # 230
Q learning, step  873 ...
Q learning, step  874 ...
Q learning, step  875 ...
Q learning, step  876 ...
Q learning, step  877 ...
Episode # 231
Q learning, step  878 ...
Q learning, step  879 ...
Q learning, step  880 ...
Q learning, step  881 ...
Q learning, step  882 ...
Q learning, step  883 ...
Q learning, step  884 ...
Q learning, step  885 ...
Q learning, step  886 ...
Q learning, step  887 ...
Q learning, step  888 ...
Q learning, step  889 ...
Episode # 232
Q learning, step  890 ...
Q learning, step  891 ...
Q learning, step  892 ...
Episode # 233
Q learning, step  893 ...
Q learning, step  89

Q learning, step  1228 ...
Q learning, step  1229 ...
Q learning, step  1230 ...
Episode # 323
Q learning, step  1231 ...
Q learning, step  1232 ...
Episode # 324
Q learning, step  1233 ...
Episode # 325
Q learning, step  1234 ...
Q learning, step  1235 ...
Q learning, step  1236 ...
Q learning, step  1237 ...
Q learning, step  1238 ...
Q learning, step  1239 ...
Q learning, step  1240 ...
Q learning, step  1241 ...
Q learning, step  1242 ...
Episode # 326
Q learning, step  1243 ...
Episode # 327
Q learning, step  1244 ...
Q learning, step  1245 ...
Q learning, step  1246 ...
Q learning, step  1247 ...
Q learning, step  1248 ...
Q learning, step  1249 ...
Q learning, step  1250 ...
Q learning, step  1251 ...
Q learning, step  1252 ...
Episode # 328
Q learning, step  1253 ...
Q learning, step  1254 ...
Q learning, step  1255 ...
Q learning, step  1256 ...
Episode # 329
Q learning, step  1257 ...
Q learning, step  1258 ...
Episode # 330
Q learning, step  1259 ...
Q learning, step  1260 .

Q learning, step  1590 ...
Q learning, step  1591 ...
Q learning, step  1592 ...
Q learning, step  1593 ...
Episode # 428
Q learning, step  1594 ...
Episode # 429
Q learning, step  1595 ...
Episode # 430
Q learning, step  1596 ...
Q learning, step  1597 ...
Q learning, step  1598 ...
Episode # 431
Q learning, step  1599 ...
Episode # 432
Q learning, step  1600 ...
Q learning, step  1601 ...
Q learning, step  1602 ...
Q learning, step  1603 ...
Q learning, step  1604 ...
Q learning, step  1605 ...
Episode # 433
Q learning, step  1606 ...
Q learning, step  1607 ...
Episode # 434
Q learning, step  1608 ...
Q learning, step  1609 ...
Episode # 435
Q learning, step  1610 ...
Q learning, step  1611 ...
Q learning, step  1612 ...
Episode # 436
Q learning, step  1613 ...
Q learning, step  1614 ...
Q learning, step  1615 ...
Q learning, step  1616 ...
Q learning, step  1617 ...
Episode # 437
Q learning, step  1618 ...
Q learning, step  1619 ...
Q learning, step  1620 ...
Q learning, step  1621 

Q learning, step  2087 ...
Q learning, step  2088 ...
Q learning, step  2089 ...
Q learning, step  2090 ...
Episode # 594
Q learning, step  2091 ...
Q learning, step  2092 ...
Episode # 595
Q learning, step  2093 ...
Q learning, step  2094 ...
Q learning, step  2095 ...
Q learning, step  2096 ...
Q learning, step  2097 ...
Q learning, step  2098 ...
Q learning, step  2099 ...
Episode # 596
Q learning, step  2100 ...
Q learning, step  2101 ...
Q learning, step  2102 ...
Q learning, step  2103 ...
Q learning, step  2104 ...
Q learning, step  2105 ...
Q learning, step  2106 ...
Episode # 597
Q learning, step  2107 ...
Episode # 598
Q learning, step  2108 ...
Q learning, step  2109 ...
Q learning, step  2110 ...
Episode # 599
Q learning, step  2111 ...
Episode # 600
Q learning, step  2112 ...
Q learning, step  2113 ...
Episode # 601
Q learning, step  2114 ...
Q learning, step  2115 ...
Episode # 602
Q learning, step  2116 ...
Q learning, step  2117 ...
Q learning, step  2118 ...
Episode # 

Q learning, step  2505 ...
Q learning, step  2506 ...
Q learning, step  2507 ...
Q learning, step  2508 ...
Q learning, step  2509 ...
Episode # 728
Q learning, step  2510 ...
Q learning, step  2511 ...
Q learning, step  2512 ...
Episode # 729
Q learning, step  2513 ...
Q learning, step  2514 ...
Q learning, step  2515 ...
Q learning, step  2516 ...
Q learning, step  2517 ...
Q learning, step  2518 ...
Episode # 730
Q learning, step  2519 ...
Episode # 731
Q learning, step  2520 ...
Q learning, step  2521 ...
Q learning, step  2522 ...
Q learning, step  2523 ...
Q learning, step  2524 ...
Q learning, step  2525 ...
Q learning, step  2526 ...
Episode # 732
Q learning, step  2527 ...
Episode # 733
Q learning, step  2528 ...
Episode # 734
Q learning, step  2529 ...
Q learning, step  2530 ...
Episode # 735
Q learning, step  2531 ...
Q learning, step  2532 ...
Q learning, step  2533 ...
Episode # 736
Q learning, step  2534 ...
Q learning, step  2535 ...
Episode # 737
Q learning, step  2536 

Q learning, step  2882 ...
Q learning, step  2883 ...
Q learning, step  2884 ...
Q learning, step  2885 ...
Q learning, step  2886 ...
Q learning, step  2887 ...
Episode # 838
Q learning, step  2888 ...
Q learning, step  2889 ...
Q learning, step  2890 ...
Q learning, step  2891 ...
Q learning, step  2892 ...
Q learning, step  2893 ...
Q learning, step  2894 ...
Q learning, step  2895 ...
Q learning, step  2896 ...
Q learning, step  2897 ...
Q learning, step  2898 ...
Q learning, step  2899 ...
Episode # 839
Q learning, step  2900 ...
Q learning, step  2901 ...
Q learning, step  2902 ...
Episode # 840
Q learning, step  2903 ...
Q learning, step  2904 ...
Episode # 841
Q learning, step  2905 ...
Episode # 842
Q learning, step  2906 ...
Q learning, step  2907 ...
Q learning, step  2908 ...
Episode # 843
Q learning, step  2909 ...
Episode # 844
Q learning, step  2910 ...
Episode # 845
Q learning, step  2911 ...
Q learning, step  2912 ...
Q learning, step  2913 ...
Q learning, step  2914 .

Q learning, step  3273 ...
Episode # 979
Q learning, step  3274 ...
Episode # 980
Q learning, step  3275 ...
Q learning, step  3276 ...
Q learning, step  3277 ...
Q learning, step  3278 ...
Episode # 981
Q learning, step  3279 ...
Q learning, step  3280 ...
Q learning, step  3281 ...
Q learning, step  3282 ...
Q learning, step  3283 ...
Q learning, step  3284 ...
Q learning, step  3285 ...
Episode # 982
Q learning, step  3286 ...
Q learning, step  3287 ...
Episode # 983
Q learning, step  3288 ...
Q learning, step  3289 ...
Episode # 984
Q learning, step  3290 ...
Episode # 985
Q learning, step  3291 ...
Q learning, step  3292 ...
Q learning, step  3293 ...
Episode # 986
Q learning, step  3294 ...
Episode # 987
Q learning, step  3295 ...
Episode # 988
Q learning, step  3296 ...
Episode # 989
Q learning, step  3297 ...
Q learning, step  3298 ...
Q learning, step  3299 ...
Q learning, step  3300 ...
Q learning, step  3301 ...
Episode # 990
Q learning, step  3302 ...
Episode # 991
Q learni

In [38]:
### Comparison of value iteration optimal policy with Q learning policy after 1000 episodes ###
diff_opt_states2 = []

for s in range(env.num_states):
    if (action_list[s] != action_list3[s]):
        diff_opt_states2.append(s)

count_diff = 0
for i in range(len(diff_opt_states2)):
    if (action_list[i] != action_list3[i]):
        print('---------Different at state ', i, '---------')
        print('Value iteration optimal policy: ',action_list[i])
        print('Q learning policy after 1000 episodes: ',action_list3[i])
        count_diff += 1

print('***************************************************************************')
print('Number of differences in policy: ', count_diff)

---------Different at state  1 ---------
Value iteration optimal policy:  DOWN
Q learning policy after 1000 episodes:  RIGHT
---------Different at state  2 ---------
Value iteration optimal policy:  RIGHT
Q learning policy after 1000 episodes:  UP
---------Different at state  4 ---------
Value iteration optimal policy:  RIGHT
Q learning policy after 1000 episodes:  LEFT
---------Different at state  5 ---------
Value iteration optimal policy:  RIGHT
Q learning policy after 1000 episodes:  DOWN
---------Different at state  6 ---------
Value iteration optimal policy:  RIGHT
Q learning policy after 1000 episodes:  UP
---------Different at state  11 ---------
Value iteration optimal policy:  DOWN
Q learning policy after 1000 episodes:  UP
---------Different at state  24 ---------
Value iteration optimal policy:  UP
Q learning policy after 1000 episodes:  DOWN
---------Different at state  25 ---------
Value iteration optimal policy:  UP
Q learning policy after 1000 episodes:  LEFT
**********

In [40]:
### Comparison of policy iteration optimal policy with Q learning policy after 1000 episodes ###
diff_opt_states3 = []

for s in range(env.num_states):
    if (action_list2[s] != action_list3[s]):
        diff_opt_states3.append(s)

count_diff = 0
for i in range(len(diff_opt_states3)):
    if (action_list2[i] != action_list3[i]):
        print('---------Different at state ', i, '---------')
        print('Policy iteration optimal policy: ',action_list2[i])
        print('Q learning policy after 1000 episodes: ',action_list3[i])
        count_diff += 1
        
print('***************************************************************************')
print('Number of differences in policy: ', count_diff)

---------Different at state  0 ---------
Policy iteration optimal policy:  UP
Q learning policy after 1000 episodes:  RIGHT
---------Different at state  1 ---------
Policy iteration optimal policy:  DOWN
Q learning policy after 1000 episodes:  RIGHT
---------Different at state  4 ---------
Policy iteration optimal policy:  UP
Q learning policy after 1000 episodes:  LEFT
---------Different at state  5 ---------
Policy iteration optimal policy:  RIGHT
Q learning policy after 1000 episodes:  DOWN
---------Different at state  6 ---------
Policy iteration optimal policy:  RIGHT
Q learning policy after 1000 episodes:  UP
---------Different at state  10 ---------
Policy iteration optimal policy:  UP
Q learning policy after 1000 episodes:  RIGHT
---------Different at state  11 ---------
Policy iteration optimal policy:  DOWN
Q learning policy after 1000 episodes:  UP
---------Different at state  20 ---------
Policy iteration optimal policy:  UP
Q learning policy after 1000 episodes:  DOWN
----

In [ ]:
### Difference between Q learning policy and optimal policies (for both algorithms) as a function of # episodes ###

# intervals of 5
eps = np.arange(1,1000,5) # np.arange includes the lower bound, excludes upper bound

value_diff = []
policy_diff = []

for e in range(len(eps)):
    temp_policy, Q, steps = q_learning(episodes=eps[e])  
    
    # need to do a conversion of temp_policy for Q-learning and value iteration policy to be a list like
    # policy iteration policy so that we can compare below
    temp_policy_convert = []
    policy_convert = []
    
    for i in range(temp_policy.shape[0]): # i.e. the rows of temp_policy = # states
        temp_policy_convert.append( temp_policy[i, :].tolist().index(1) )
        policy_convert.append( policy[i,:].tolist().index(1) )
    
    value_diff.append(np.linalg.norm( np.asarray(temp_policy_convert) - np.asarray(policy_convert) ))
    policy_diff.append(np.linalg.norm( np.asarray(temp_policy_convert) - np.asarray(policy2) ))

Episode # 0
Episode # 0
Episode # 1
Episode # 2
Episode # 3
Episode # 4
Episode # 5
Episode # 0
Episode # 1
Episode # 2
Episode # 3
Episode # 4
Episode # 5
Episode # 6
Episode # 7
Episode # 8
Episode # 9
Episode # 10
Episode # 0
Episode # 1
Episode # 2
Episode # 3
Episode # 4
Episode # 5
Episode # 6
Episode # 7
Episode # 8
Episode # 9
Episode # 10
Episode # 11
Episode # 12
Episode # 13
Episode # 14
Episode # 15
Episode # 0
Episode # 1
Episode # 2
Episode # 3
Episode # 4
Episode # 5
Episode # 6
Episode # 7
Episode # 8
Episode # 9
Episode # 10
Episode # 11
Episode # 12
Episode # 13
Episode # 14
Episode # 15
Episode # 16
Episode # 17
Episode # 18
Episode # 19
Episode # 20
Episode # 0
Episode # 1
Episode # 2
Episode # 3
Episode # 4
Episode # 5
Episode # 6
Episode # 7
Episode # 8
Episode # 9
Episode # 10
Episode # 11
Episode # 12
Episode # 13
Episode # 14
Episode # 15
Episode # 16
Episode # 17
Episode # 18
Episode # 19
Episode # 20
Episode # 21
Episode # 22
Episode # 23
Episode # 24
Episode

Episode # 0
Episode # 1
Episode # 2
Episode # 3
Episode # 4
Episode # 5
Episode # 6
Episode # 7
Episode # 8
Episode # 9
Episode # 10
Episode # 11
Episode # 12
Episode # 13
Episode # 14
Episode # 15
Episode # 16
Episode # 17
Episode # 18
Episode # 19
Episode # 20
Episode # 21
Episode # 22
Episode # 23
Episode # 24
Episode # 25
Episode # 26
Episode # 27
Episode # 28
Episode # 29
Episode # 30
Episode # 31
Episode # 32
Episode # 33
Episode # 34
Episode # 35
Episode # 36
Episode # 37
Episode # 38
Episode # 39
Episode # 40
Episode # 41
Episode # 42
Episode # 43
Episode # 44
Episode # 45
Episode # 46
Episode # 47
Episode # 48
Episode # 49
Episode # 50
Episode # 51
Episode # 52
Episode # 53
Episode # 54
Episode # 55
Episode # 56
Episode # 57
Episode # 58
Episode # 59
Episode # 60
Episode # 61
Episode # 62
Episode # 63
Episode # 64
Episode # 65
Episode # 66
Episode # 67
Episode # 68
Episode # 69
Episode # 70
Episode # 71
Episode # 72
Episode # 73
Episode # 74
Episode # 75
Episode # 76
Episode #

Episode # 84
Episode # 85
Episode # 86
Episode # 87
Episode # 88
Episode # 89
Episode # 90
Episode # 91
Episode # 92
Episode # 93
Episode # 94
Episode # 95
Episode # 96
Episode # 97
Episode # 98
Episode # 99
Episode # 100
Episode # 101
Episode # 102
Episode # 103
Episode # 104
Episode # 105
Episode # 106
Episode # 107
Episode # 108
Episode # 109
Episode # 110
Episode # 111
Episode # 112
Episode # 113
Episode # 114
Episode # 115
Episode # 0
Episode # 1
Episode # 2
Episode # 3
Episode # 4
Episode # 5
Episode # 6
Episode # 7
Episode # 8
Episode # 9
Episode # 10
Episode # 11
Episode # 12
Episode # 13
Episode # 14
Episode # 15
Episode # 16
Episode # 17
Episode # 18
Episode # 19
Episode # 20
Episode # 21
Episode # 22
Episode # 23
Episode # 24
Episode # 25
Episode # 26
Episode # 27
Episode # 28
Episode # 29
Episode # 30
Episode # 31
Episode # 32
Episode # 33
Episode # 34
Episode # 35
Episode # 36
Episode # 37
Episode # 38
Episode # 39
Episode # 40
Episode # 41
Episode # 42
Episode # 43
Episod

Episode # 118
Episode # 119
Episode # 120
Episode # 121
Episode # 122
Episode # 123
Episode # 124
Episode # 125
Episode # 126
Episode # 127
Episode # 128
Episode # 129
Episode # 130
Episode # 131
Episode # 132
Episode # 133
Episode # 134
Episode # 135
Episode # 136
Episode # 137
Episode # 138
Episode # 139
Episode # 140
Episode # 141
Episode # 142
Episode # 143
Episode # 144
Episode # 145
Episode # 0
Episode # 1
Episode # 2
Episode # 3
Episode # 4
Episode # 5
Episode # 6
Episode # 7
Episode # 8
Episode # 9
Episode # 10
Episode # 11
Episode # 12
Episode # 13
Episode # 14
Episode # 15
Episode # 16
Episode # 17
Episode # 18
Episode # 19
Episode # 20
Episode # 21
Episode # 22
Episode # 23
Episode # 24
Episode # 25
Episode # 26
Episode # 27
Episode # 28
Episode # 29
Episode # 30
Episode # 31
Episode # 32
Episode # 33
Episode # 34
Episode # 35
Episode # 36
Episode # 37
Episode # 38
Episode # 39
Episode # 40
Episode # 41
Episode # 42
Episode # 43
Episode # 44
Episode # 45
Episode # 46
Episode

Episode # 163
Episode # 164
Episode # 165
Episode # 166
Episode # 167
Episode # 168
Episode # 169
Episode # 170
Episode # 0
Episode # 1
Episode # 2
Episode # 3
Episode # 4
Episode # 5
Episode # 6
Episode # 7
Episode # 8
Episode # 9
Episode # 10
Episode # 11
Episode # 12
Episode # 13
Episode # 14
Episode # 15
Episode # 16
Episode # 17
Episode # 18
Episode # 19
Episode # 20
Episode # 21
Episode # 22
Episode # 23
Episode # 24
Episode # 25
Episode # 26
Episode # 27
Episode # 28
Episode # 29
Episode # 30
Episode # 31
Episode # 32
Episode # 33
Episode # 34
Episode # 35
Episode # 36
Episode # 37
Episode # 38
Episode # 39
Episode # 40
Episode # 41
Episode # 42
Episode # 43
Episode # 44
Episode # 45
Episode # 46
Episode # 47
Episode # 48
Episode # 49
Episode # 50
Episode # 51
Episode # 52
Episode # 53
Episode # 54
Episode # 55
Episode # 56
Episode # 57
Episode # 58
Episode # 59
Episode # 60
Episode # 61
Episode # 62
Episode # 63
Episode # 64
Episode # 65
Episode # 66
Episode # 67
Episode # 68
E

...

Episode # 257
Episode # 258
Episode # 259
Episode # 260
Episode # 261
Episode # 262
Episode # 263
Episode # 264
Episode # 265
Episode # 266
Episode # 267
Episode # 268
Episode # 269
Episode # 270
Episode # 271
Episode # 272
Episode # 273
Episode # 274
Episode # 275
Episode # 276
Episode # 277
Episode # 278
Episode # 279
Episode # 280
Episode # 281
Episode # 282
Episode # 283
Episode # 284
Episode # 285
Episode # 286
Episode # 287
Episode # 288
Episode # 289
Episode # 290
Episode # 291
Episode # 292
Episode # 293
Episode # 294
Episode # 295
Episode # 296
Episode # 297
Episode # 298
Episode # 299
Episode # 300
Episode # 301
Episode # 302
Episode # 303
Episode # 304
Episode # 305
Episode # 306
Episode # 307
Episode # 308
Episode # 309
Episode # 310
Episode # 311
Episode # 312
Episode # 313
Episode # 314
Episode # 315
Episode # 316
Episode # 317
Episode # 318
Episode # 319
Episode # 320
Episode # 321
Episode # 322
Episode # 323
Episode # 324
Episode # 325
Episode # 326
Episode # 327
Episod

Episode # 167
Episode # 168
Episode # 169
Episode # 170
Episode # 171
Episode # 172
Episode # 173
Episode # 174
Episode # 175
Episode # 176
Episode # 177
Episode # 178
Episode # 179
Episode # 180
Episode # 181
Episode # 182
Episode # 183
Episode # 184
Episode # 185
Episode # 186
Episode # 187
Episode # 188
Episode # 189
Episode # 190
Episode # 191
Episode # 192
Episode # 193
Episode # 194
Episode # 195
Episode # 196
Episode # 197
Episode # 198
Episode # 199
Episode # 200
Episode # 201
Episode # 202
Episode # 203
Episode # 204
Episode # 205
Episode # 206
Episode # 207
Episode # 208
Episode # 209
Episode # 210
Episode # 211
Episode # 212
Episode # 213
Episode # 214
Episode # 215
Episode # 216
Episode # 217
Episode # 218
Episode # 219
Episode # 220
Episode # 221
Episode # 222
Episode # 223
Episode # 224
Episode # 225
Episode # 226
Episode # 227
Episode # 228
Episode # 229
Episode # 230
Episode # 231
Episode # 232
Episode # 233
Episode # 234
Episode # 235
Episode # 236
Episode # 237
Episod

Episode # 154
Episode # 155
Episode # 156
Episode # 157
Episode # 158
Episode # 159
Episode # 160
Episode # 161
Episode # 162
Episode # 163
Episode # 164
Episode # 165
Episode # 166
Episode # 167
Episode # 168
Episode # 169
Episode # 170
Episode # 171
Episode # 172
Episode # 173
Episode # 174
Episode # 175
Episode # 176
Episode # 177
Episode # 178
Episode # 179
Episode # 180
Episode # 181
Episode # 182
Episode # 183
Episode # 184
Episode # 185
Episode # 186
Episode # 187
Episode # 188
Episode # 189
Episode # 190
Episode # 191
Episode # 192
Episode # 193
Episode # 194
Episode # 195
Episode # 196
Episode # 197
Episode # 198
Episode # 199
Episode # 200
Episode # 201
Episode # 202
Episode # 203
Episode # 204
Episode # 205
Episode # 206
Episode # 207
Episode # 208
Episode # 209
Episode # 210
Episode # 211
Episode # 212
Episode # 213
Episode # 214
Episode # 215
Episode # 216
Episode # 217
Episode # 218
Episode # 219
Episode # 220
Episode # 221
Episode # 222
Episode # 223
Episode # 224
Episod

Episode # 885
Episode # 886
Episode # 887
Episode # 888
Episode # 889
Episode # 890
Episode # 891
Episode # 892
Episode # 893
Episode # 894
Episode # 895
Episode # 0
Episode # 1
Episode # 2
Episode # 3
Episode # 4
Episode # 5
Episode # 6
Episode # 7
Episode # 8
Episode # 9
Episode # 10
Episode # 11
Episode # 12
Episode # 13
Episode # 14
Episode # 15
Episode # 16
Episode # 17
Episode # 18
Episode # 19
Episode # 20
Episode # 21
Episode # 22
Episode # 23
Episode # 24
Episode # 25
Episode # 26
Episode # 27
Episode # 28
Episode # 29
Episode # 30
Episode # 31
Episode # 32
Episode # 33
Episode # 34
Episode # 35
Episode # 36
Episode # 37
Episode # 38
Episode # 39
Episode # 40
Episode # 41
Episode # 42
Episode # 43
Episode # 44
Episode # 45
Episode # 46
Episode # 47
Episode # 48
Episode # 49
Episode # 50
Episode # 51
Episode # 52
Episode # 53
Episode # 54
Episode # 55
Episode # 56
Episode # 57
Episode # 58
Episode # 59
Episode # 60
Episode # 61
Episode # 62
Episode # 63
Episode # 64
Episode # 6

Episode # 724
Episode # 725
Episode # 726
Episode # 727
Episode # 728
Episode # 729
Episode # 730
Episode # 731
Episode # 732
Episode # 733
Episode # 734
Episode # 735
Episode # 736
Episode # 737
Episode # 738
Episode # 739
Episode # 740
Episode # 741
Episode # 742
Episode # 743
Episode # 744
Episode # 745
Episode # 746
Episode # 747
Episode # 748
Episode # 749
Episode # 750
Episode # 751
Episode # 752
Episode # 753
Episode # 754
Episode # 755
Episode # 756
Episode # 757
Episode # 758
Episode # 759
Episode # 760
Episode # 761
Episode # 762
Episode # 763
Episode # 764
Episode # 765
Episode # 766
Episode # 767
Episode # 768
Episode # 769
Episode # 770
Episode # 771
Episode # 772
Episode # 773
Episode # 774
Episode # 775
Episode # 776
Episode # 777
Episode # 778
Episode # 779
Episode # 780
Episode # 781
Episode # 782
Episode # 783
Episode # 784
Episode # 785
Episode # 786
Episode # 787
Episode # 788
Episode # 789
Episode # 790
Episode # 791
Episode # 792
Episode # 793
Episode # 794
Episod

In [ ]:
import matplotlib.pyplot as plt

plt.plot(eps, value_diff, 'r', eps, policy_diff, 'b')
plt.xlabel('# episodes')
plt.ylabel('euclidean distance between optimal policy and q-learning policy')
plt.xticks(np.arange(1, 1000, 100))
# lines.legend(['Value iteration', 'Policy iteration'])
plt.show()